<a href="https://colab.research.google.com/github/sunnys-lab/xray_classification_with_tpus/blob/main/medical_image_classification_20220628.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>